In [10]:
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
train_df = pd.read_csv("/Users/hansuho/Desktop/Dacon/ParkingDemandPrediction/235745_parking_data/train.csv")

In [51]:
train_df.head(2)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0


In [52]:
train_df.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [53]:
train_df.rename(columns={'단지코드': 'complex_cd', '총세대수': 'n_house', '임대건물구분': 'bldg_type', '지역': 'regions', '공급유형': 'supply_type',
                        '전용면적': 'area', '전용면적별세대수': 'n_house_area', '공가수': 'n_empty_house', '자격유형': 'q_type', '임대보증금': 'lease_deposit',
                        '임대료': 'lease_price', "도보 10분거리 내 지하철역 수(환승노선 수 반영)": 'n_subway', '도보 10분거리 내 버스정류장 수': 'n_bus',
                        '단지내주차면수': 'parking_area', '등록차량수': 'cars'}, inplace = True)

In [54]:
train_df.head()

,complex_cd,n_house,bldg_type,regions,supply_type,area,n_house_area,n_empty_house,q_type,lease_deposit,lease_price,n_subway,n_bus,parking_area,cars
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [37]:
norminal_cols = ['complex_cd', 'bldg_type', 'region', 'supply_type', 'q_type']

print(f"unique 단지코드 개수: {train_df['complex_cd'].nunique()}")
print()
for col in norminal_cols[1:]:
    print(f"{col}: {train_df[col].nunique()}")
    print(train_df[col].unique())
    print()

unique 단지코드 개수: 423

bldg_type: 2
['아파트' '상가']

region: 1
[0]

supply_type: 10
['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']

q_type: 15
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']



In [55]:
# 전용면적 5의 배수로 변경, 15-100 상하한 지정
train_df['area'] = train_df['area']//5*5

idx = train_df[train_df['area'] > 100].index
train_df.loc[idx, 'area'] = 100
idx = train_df[train_df['area'] < 15].index
train_df.loc[idx, 'area'] = 15

In [56]:
# 지역명 숫자로 변경
regions_dict = {}
for i, region in enumerate(train_df['regions'].unique()):
    print(i, region)
    regions_dict[region] = i

train_df['regions'] = train_df['regions'].map(regions)

0 경상북도
1 경상남도
2 대전광역시
3 경기도
4 전라북도
5 강원도
6 광주광역시
7 충청남도
8 부산광역시
9 제주특별자치도
10 울산광역시
11 충청북도
12 전라남도
13 대구광역시
14 서울특별시
15 세종특별자치시


In [40]:
regions

{0: 0}

In [ ]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i